# DeepLearning

## MNIST Dataset
The MNIST database is a well-known academic dataset used to benchmark
classification performance. The data consists of 60,000 training images and
10,000 test images. Each image is a standardized $28^2$ pixel greyscale image of
a single handwritten digit. An example of the scanned handwritten digits is
shown
![Example MNIST digit images](../../h2o-docs/src/booklets/v2_2015/source/images/mnistdigits.jpg)

In [8]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,20 hours 37 mins
H2O cluster version:,3.11.0.99999
H2O cluster version age:,1 day
H2O cluster name:,fmilo
H2O cluster total nodes:,1
H2O cluster free memory:,12.67 Gb
H2O cluster total cores:,40
H2O cluster allowed cores:,40
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [13]:
test_df = h2o.import_file("https://h2o-public-test-data.s3.amazonaws.com/bigdata/laptop/mnist/test.csv.gz")

Parse progress: |█████████████████████████████████████████████████████████████████████████████| 100%


In [12]:
train_df = h2o.import_file("https://h2o-public-test-data.s3.amazonaws.com/bigdata/laptop/mnist/train.csv.gz")

Parse progress: |█████████████████████████████████████████████████████████████████████████████| 100%


In [14]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

Specify the response and predictor columns

In [17]:
y = "C785"
x = train_df.names[0:784]

In [33]:
train_df[y] = train_df[y].asfactor()
test_df[y] = test_df[y].asfactor()

Train Deep Learning model and validate on test set

In [34]:
model = H2ODeepLearningEstimator(
   distribution="multinomial",
   activation="RectifierWithDropout",
   hidden=[32, 32, 32],
   input_dropout_ratio=0.2,
   sparse=True,
   l1=1e-5,
   epochs=10)

In [36]:
model.train(
    x=x, 
    y=y,
    training_frame=train_df,
    validation_frame=test_df
)

deeplearning Model Build progress: |██████████████████████████████████████████████████████████| 100%


## Using Crossvalidation

If the value specified for nfolds is a positive integer, N-fold cross-validation is
performed on the training frame and the cross-validation metrics are computed
and stored as model output. 

To disable cross-validation, use `nfolds=0`, which is the default value.

Advanced users can also specify a fold column that defines the holdout
fold associated with each row. By default, the holdout fold assignment is
random. H2O supports other schemes such as round-robin assignment using the modulo
operator.

Perform 5-fold cross-validation on training_frame

In [42]:
model_crossvalidated = H2ODeepLearningEstimator(
    distribution="multinomial",
    activation="RectifierWithDropout",
    hidden=[32,32,32],
    input_dropout_ratio=0.2,
    sparse=True,
    l1=1e-5,
    epochs=10,
    nfolds=5
)

In [45]:
model_crossvalidated.train(
    x=x,
    y=y,
    training_frame=train_df
)

deeplearning Model Build progress: |██████████████████████████████████████████████████████████| 100%


## Extracting and Handling the Results

We can now extract the parameters of our model, examine the scoring process,
and make predictions on new data.

In [47]:
# View specified parameters of the Deep Learning model
model.params;

In [48]:
# Examine the performance of the trained model
model # display all performance metrics

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_model_python_1475860541731_14
Status of Neuron Layers: predicting C785, 10-class classification, multinomial distribution, CrossEntropy loss, 25,418 weights/biases, 406.4 KB, 600,000 training samples, mini-batch size 1



,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
,1,717,Input,20.0,,,,,,,,,
,2,32,RectifierDropout,50.0,1e-05,0.0,0.0300522,0.1737258,0.0,-0.0081575,0.0688910,0.4565871,0.1563672
,3,32,RectifierDropout,50.0,1e-05,0.0,0.0003242,0.0001677,0.0,-0.0298821,0.2032132,0.7023755,0.3328817
,4,32,RectifierDropout,50.0,1e-05,0.0,0.0005160,0.0003185,0.0,-0.0561277,0.2058569,0.7590021,0.4316647
,5,10,Softmax,,1e-05,0.0,0.0024298,0.0019817,0.0,-0.4316639,1.0057969,-1.8806262,1.5657716




ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.127232091567
RMSE: 0.356696077308
LogLoss: 0.423974904018
Mean Per-Class Error: 0.100728737967
Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
1007.0,9.0,0.0,1.0,3.0,9.0,3.0,0.0,9.0,1.0,0.0335893,"35 / 1,042"
0.0,1148.0,1.0,2.0,0.0,0.0,1.0,1.0,4.0,2.0,0.0094909,"11 / 1,159"
10.0,40.0,846.0,42.0,10.0,10.0,8.0,8.0,28.0,3.0,0.1582090,"159 / 1,005"
1.0,15.0,13.0,925.0,0.0,75.0,1.0,16.0,6.0,4.0,0.1240530,"131 / 1,056"
0.0,43.0,1.0,0.0,844.0,1.0,2.0,1.0,5.0,55.0,0.1134454,108 / 952
4.0,8.0,5.0,13.0,1.0,831.0,5.0,5.0,23.0,2.0,0.0735786,66 / 897
5.0,34.0,3.0,0.0,7.0,15.0,921.0,0.0,9.0,0.0,0.0734406,73 / 994
1.0,32.0,4.0,4.0,4.0,2.0,0.0,975.0,1.0,32.0,0.0758294,"80 / 1,055"
3.0,134.0,4.0,15.0,0.0,46.0,1.0,2.0,713.0,6.0,0.2283550,211 / 924
4.0,34.0,0.0,6.0,16.0,17.0,0.0,38.0,3.0,888.0,0.1172962,"118 / 1,006"


Top-10 Hit Ratios: 


...

0,1,2,3,4,5,6,7,8,9,Error,Rate
959.0,3.0,0.0,1.0,0.0,8.0,5.0,3.0,1.0,0.0,0.0214286,21 / 980
0.0,1121.0,1.0,4.0,0.0,2.0,3.0,0.0,4.0,0.0,0.0123348,"14 / 1,135"
10.0,42.0,854.0,52.0,13.0,10.0,4.0,9.0,36.0,2.0,0.1724806,"178 / 1,032"
2.0,12.0,8.0,903.0,0.0,62.0,0.0,15.0,6.0,2.0,0.1059406,"107 / 1,010"
2.0,58.0,3.0,0.0,847.0,0.0,7.0,2.0,1.0,62.0,0.1374745,135 / 982
4.0,8.0,2.0,19.0,2.0,821.0,5.0,10.0,18.0,3.0,0.0795964,71 / 892
9.0,27.0,6.0,0.0,9.0,24.0,879.0,0.0,4.0,0.0,0.0824635,79 / 958
1.0,40.0,5.0,8.0,3.0,1.0,0.0,946.0,1.0,23.0,0.0797665,"82 / 1,028"
6.0,105.0,3.0,11.0,3.0,41.0,3.0,7.0,789.0,6.0,0.1899384,185 / 974
6.0,39.0,1.0,7.0,17.0,15.0,0.0,22.0,3.0,899.0,0.1090188,"110 / 1,009"


Top-10 Hit Ratios: 


k,hit_ratio
1,0.9018
2,0.9518
3,0.9669
4,0.9769
5,0.9854000
6,0.9925000
7,0.9962999
8,0.9981999
9,0.9994000
10,0.9999999


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
,2016-10-08 07:09:25,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan,nan,nan
,2016-10-08 07:09:27,4.490 sec,79893 obs/sec,1.0,1,60000.0,0.5737880,0.9128070,0.2784936,0.5780449,0.9269061,0.2857
,2016-10-08 07:09:32,9.961 sec,104748 obs/sec,10.0,10,600000.0,0.3566961,0.4239749,0.0983152,0.3621718,0.4397471,0.0982


In [50]:
model.model_performance(train=True) # training metrics
model.model_performance(valid=True) # validation metrics


ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.131168433004
RMSE: 0.362171828009
LogLoss: 0.439747110234
Mean Per-Class Error: 0.0990442773113
Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
959.0,3.0,0.0,1.0,0.0,8.0,5.0,3.0,1.0,0.0,0.0214286,21 / 980
0.0,1121.0,1.0,4.0,0.0,2.0,3.0,0.0,4.0,0.0,0.0123348,"14 / 1,135"
10.0,42.0,854.0,52.0,13.0,10.0,4.0,9.0,36.0,2.0,0.1724806,"178 / 1,032"
2.0,12.0,8.0,903.0,0.0,62.0,0.0,15.0,6.0,2.0,0.1059406,"107 / 1,010"
2.0,58.0,3.0,0.0,847.0,0.0,7.0,2.0,1.0,62.0,0.1374745,135 / 982
4.0,8.0,2.0,19.0,2.0,821.0,5.0,10.0,18.0,3.0,0.0795964,71 / 892
9.0,27.0,6.0,0.0,9.0,24.0,879.0,0.0,4.0,0.0,0.0824635,79 / 958
1.0,40.0,5.0,8.0,3.0,1.0,0.0,946.0,1.0,23.0,0.0797665,"82 / 1,028"
6.0,105.0,3.0,11.0,3.0,41.0,3.0,7.0,789.0,6.0,0.1899384,185 / 974
6.0,39.0,1.0,7.0,17.0,15.0,0.0,22.0,3.0,899.0,0.1090188,"110 / 1,009"


Top-10 Hit Ratios: 


k,hit_ratio
1,0.9018
2,0.9518
3,0.9669
4,0.9769
5,0.9854000
6,0.9925000
7,0.9962999
8,0.9981999
9,0.9994000
10,0.9999999


The validation error is based on the
parameter `score validation samples`, which configures the same value
on the validation set (by default, this is the entire validation set).

In [51]:
# Get MSE only
model.mse(valid=True)

0.1311684330036781

In [53]:
# Cross-validated MSE
model_crossvalidated.mse(xval=True)

0.12221266388266913

In [5]:
ls ../../h2o-docs/src/booklets/v2_2015/source/images/

mnistdigits.jpg  scurve.png  waves.png


## Predicting

Once we have a satisfactory model (as determined by the validation or crossvalidation
metrics), use the `h2o.predict()` command to compute and store
predictions on new data for additional refinements in the interactive data science
process.

In [54]:
predictions = model.predict(test_df)

deeplearning prediction progress: |███████████████████████████████████████████████████████████| 100%


## Variable Importance

Variable importance allows us to view the absolute and relative predictive strength of
each feature in the prediction task.
Each H2O algorithm class has its own methodology for computing variable importance.

You can enable the variable importance, by setting the `variable_importances` parameter to `True`.

H2O’s Deep Learning uses the Gedeon method [Gedeon, 1997](http://users.cecs.anu.edu.au/~Tom.Gedeon/pdfs/ContribDataMinv2.pdf), which is disabled
by default since it can be slow for large networks. 

If variable importance is a top priority in your analysis, consider training a Distributed Random Forest (DRF) model and compare the generated variable importances.

In [65]:
# Train Deep Learning model and validate on test set and save the variable importances
model_variable_importances = H2ODeepLearningEstimator(
     distribution="multinomial",
     activation="RectifierWithDropout",
     hidden=[32,32,32],
     input_dropout_ratio=0.2,
     sparse=True,
     l1=1e-5,
     epochs=10,
     variable_importances=True)

In [67]:
model_variable_importances.train(
         x=x,
         y=y,
         training_frame=train_df,
         validation_frame=test_df)

deeplearning Model Build progress: |██████████████████████████████████████████████████████████| 100%


In [70]:
import pandas as pd

In [71]:
# Retrieve the variable importance
pd.DataFrame(model_variable_importances.varimp())

,0,1,2,3
0,C516,1.000000,1.000000,0.002653
1,C517,0.989558,0.989558,0.002626
2,C487,0.979800,0.979800,0.002600
3,C352,0.977987,0.977987,0.002595
4,C489,0.965509,0.965509,0.002562
5,C376,0.953370,0.953370,0.002530
6,C490,0.952001,0.952001,0.002526
7,C515,0.947224,0.947224,0.002513
8,C378,0.946891,0.946891,0.002513
9,C348,0.945993,0.945993,0.002510


## Model Comparison with Grid Search

Grid search provides more subtle insights into the model tuning and selection
process by inspecting and comparing our trained models after the grid search process is complete. 

To learn when and how to select different parameter
configurations in a grid search, refer to Parameters for parameter descriptions
and configurable values.

There are different strategies to explore the hyperparameter combinatorial space:

- Cartesian Search: test *every* single combination
- Random Search: sample combinations

## Cartesian Search
In this example, three different network topologies and two different $l_1$ norm
weights are specified. This grid search model trains six different models using all
possible combinations of these parameters; other parameter combinations can
be specified for a larger space of models. Note that the models will most likely
converge before the default value of epochs, since early stopping is enabled.

In [72]:
from h2o.grid.grid_search import H2OGridSearch

In [73]:
hyper_parameters = {
    "hidden":[[32,32],[32,16,8],[100]], 
    "l1":[1e-4,1e-3],
}

model_grid = H2OGridSearch(H2ODeepLearningEstimator, hyper_params=hyper_parameters)

In [76]:
model_grid.train(
    x=x, 
    y=y,
    distribution="multinomial", 
    epochs=1000,
    training_frame=train_df, 
    validation_frame=test_df,
    score_interval=2, 
    stopping_rounds=3,
    stopping_tolerance=0.05,
    stopping_metric="misclassification")

deeplearning Grid Build progress: |███████████████████████████████████████████████████████████| 100%


In [87]:
# print model grid search results
model_grid

Grid Search of Model H2ODeepLearningEstimator: 


Model,"Hyperparameters: [hidden, l1]"
1,"[[17, 32], 1e-06]"
2,"[[17, 32], 2e-06]"
3,"[[17, 32], 3e-06]"
4,"[[17, 32], 4e-06]"
5,"[[17, 32], 5e-06]"
---,---
4996,"[[10, 10, 10, 10], 0.000996]"
4997,"[[10, 10, 10, 10], 0.000997]"
4998,"[[10, 10, 10, 10], 0.000998]"
4999,"[[10, 10, 10, 10], 0.000999]"



See the whole table with table.as_data_frame()



In [88]:
for model in model_grid:
    print model.model_id + " mse: " + str(model.mse())

TypeError: object of type 'NoneType' has no len()

In [85]:
import pandas as pd

In [86]:
grid_results = pd.DataFrame([[m.model_id, m.mse] for m in model_grid])

TypeError: object of type 'NoneType' has no len()

## Random Grid Search

If the search space is too large you can let the GridSearch algorithm select the parameter, by sampling from the parameter space. 

Just specify how many models (and/or how much training time) you want, and provide a seed to make the random selection deterministic.

In [89]:
hyper_parameters = {
    "hidden":[[17,32],[8,19],[32,16,8],[100],[10,10,10,10]], 
    "l1":[s/1e6 for s in range(1,1001)]
}

In [90]:
search_criteria = {"strategy":"RandomDiscrete",
"max_models":10, "max_runtime_secs":100,
 "seed":123456}

model_grid_random_search = H2OGridSearch(H2ODeepLearningEstimator,
    hyper_params=hyper_parameters,
    search_criteria=search_criteria)

In [91]:
model_grid_random_search.train(x=x, y=y,
                distribution="multinomial", 
                epochs=1000,
                training_frame=train_df, 
                validation_frame=test_df,
                score_interval=2, 
                stopping_rounds=3,
                stopping_tolerance=0.05,
                stopping_metric="misclassification")

deeplearning Grid Build progress: |███████████████████████████████████████████████████████████| 100%


In [93]:
grid_results = pd.DataFrame([[m.model_id, m.mse] for m in model_grid_random_search])

In [95]:
grid_results

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  Grid_DeepLearning_py_6_sid_88bf_model_python_1475860541731_19_model_0
Status of Neuron Layers: predicting C785, 10-class classification, multinomial distribution, CrossEntropy loss, 23,730 weights/biases, 386.1 KB, 3,401,550 training samples, mini-batch size 1



,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
,1,717,Input,0.0,,,,,,,,,
,2,32,Rectifier,0.0,0.000385,0.0,0.0260866,0.0597005,0.0,0.0000871,0.0337432,-0.1114010,0.2024258
,3,16,Rectifier,0.0,0.000385,0.0,0.0004366,0.0002817,0.0,-0.0267806,0.1979921,0.6024806,0.2942238
,4,8,Rectifier,0.0,0.000385,0.0,0.0007993,0.0004057,0.0,0.1118247,0.4735895,0.5261426,0.2169312
,5,10,Softmax,,0.000385,0.0,0.0034330,0.0050450,0.0,-1.5000857,1.6935296,-0.2194734,0.5196128




ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.0178051323747
RMSE: 0.133435873642
LogLoss: 0.0692677321334
Mean Per-Class Error: 0.0206264534176
Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
921.0,0.0,3.0,0.0,2.0,1.0,3.0,0.0,3.0,1.0,0.0139186,13 / 934
0.0,1099.0,3.0,2.0,1.0,1.0,0.0,0.0,5.0,0.0,0.0108011,"12 / 1,111"
3.0,1.0,956.0,7.0,3.0,0.0,1.0,3.0,4.0,0.0,0.0224949,22 / 978
0.0,0.0,9.0,1015.0,0.0,4.0,0.0,2.0,9.0,2.0,0.0249760,"26 / 1,041"
0.0,4.0,0.0,1.0,970.0,1.0,5.0,1.0,0.0,10.0,0.0221774,22 / 992
2.0,1.0,2.0,6.0,0.0,829.0,4.0,1.0,5.0,1.0,0.0258519,22 / 851
4.0,1.0,0.0,0.0,3.0,2.0,1029.0,0.0,5.0,0.0,0.0143678,"15 / 1,044"
0.0,2.0,2.0,6.0,0.0,6.0,0.0,1021.0,2.0,2.0,0.0192123,"20 / 1,041"
8.0,1.0,2.0,4.0,0.0,1.0,0.0,0.0,940.0,5.0,0.0218522,21 / 961
6.0,0.0,0.0,2.0,5.0,4.0,0.0,6.0,7.0,950.0,0.0306122,30 / 980


Top-10 Hit Ratios: 


...

Top-10 Hit Ratios: 


k,hit_ratio
1,0.9588
2,0.9884000
3,0.9959000
4,0.9981000
5,0.999
6,0.9994
7,0.9997
8,1.0
9,1.0
10,1.0


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
,2016-10-08 08:20:14,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan,nan,nan
,2016-10-08 08:20:20,1 min 33.354 sec,17709 obs/sec,1.6673833,1,100043.0,0.1683309,0.1240016,0.0321005,0.1929320,0.1667586,0.0445
,2016-10-08 08:20:32,1 min 44.986 sec,18232 obs/sec,5.00415,3,300249.0,0.1560327,0.0919373,0.0278138,0.1842158,0.1313934,0.0412


,0,1
0,Grid_DeepLearning_py_6_sid_88bf_model_python_1475860541731_19_model_0,<bound method H2ODeepLearningEstimator.mse of >
1,Grid_DeepLearning_py_6_sid_88bf_model_python_1475860541731_19_model_1,<bound method H2ODeepLearningEstimator.mse of >


## Model Checkpoints 



H2O supporst model checkpoints. You can store the `state` of training and resume it later.
Checkpointing can be used to reload existing models that were saved to
disk in a previous session. 

To resume model training, use checkpoint model keys (model id) to incrementally
train a specific model using more iterations, more data, different data, and
so forth. To further train the initial model, use it (or its key) as a checkpoint
argument for a new model.

To improve this initial model, start from the previous model and add iterations by
building another model, specifying checkpoint=previous model id, and
changing train samples per iteration, target ratio comm to comp,
or other parameters. Many parameters can be changed between checkpoints,
especially those that affect regularization or performance tuning.

You can use GridSearch with checkpoint restarts to scan a broader range of hyperparameter combinations.

In [97]:
# Re-start the training process on a saved DL model using the ‘checkpoint‘ argument
model_checkpoint = H2ODeepLearningEstimator(
     checkpoint=model,
     distribution="multinomial",
     activation="RectifierWithDropout",
     hidden=[32,32,32],
     input_dropout_ratio=0.2,
     sparse=True,
     l1=1e-5,
     epochs=20)

In [98]:
model_checkpoint.train(
 x=x,
 y=y,
 training_frame=train_df,
 validation_frame=test_df)

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  Grid_DeepLearning_py_6_sid_88bf_model_python_1475860541731_17_model_1
Status of Neuron Layers: predicting C785, 10-class classification, multinomial distribution, CrossEntropy loss, 23,730 weights/biases, 386.1 KB, 1,900,147 training samples, mini-batch size 1



,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
,1,717,Input,0.0,,,,,,,,,
,2,32,Rectifier,0.0,0.0001,0.0,0.0437768,0.0980242,0.0,0.0025427,0.0571122,0.2771530,0.1923100
,3,16,Rectifier,0.0,0.0001,0.0,0.0003659,0.0002226,0.0,-0.0066047,0.2078319,0.9822588,0.0885558
,4,8,Rectifier,0.0,0.0001,0.0,0.0003314,0.0001826,0.0,0.0068420,0.3106947,0.9466416,0.0708897
,5,10,Softmax,,0.0001,0.0,0.0036559,0.0037964,0.0,-0.7710395,1.5119662,-0.2692554,0.2979772




ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.0169759190407
RMSE: 0.130291669115
LogLoss: 0.0645206363351
Mean Per-Class Error: 0.021345160565
Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
948.0,0.0,3.0,0.0,0.0,1.0,3.0,0.0,0.0,1.0,0.0083682,8 / 956
0.0,1107.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0044964,"5 / 1,112"
6.0,1.0,978.0,2.0,2.0,0.0,2.0,5.0,0.0,0.0,0.0180723,18 / 996
1.0,0.0,8.0,943.0,2.0,12.0,0.0,5.0,4.0,5.0,0.0377551,37 / 980
0.0,4.0,2.0,0.0,979.0,0.0,1.0,2.0,1.0,8.0,0.0180542,18 / 997
0.0,1.0,1.0,8.0,1.0,826.0,5.0,0.0,2.0,6.0,0.0282353,24 / 850
8.0,0.0,0.0,0.0,0.0,2.0,959.0,0.0,0.0,0.0,0.0103199,10 / 969
0.0,2.0,4.0,1.0,2.0,0.0,0.0,1087.0,0.0,3.0,0.0109190,"12 / 1,099"
6.0,4.0,3.0,5.0,0.0,8.0,1.0,1.0,910.0,9.0,0.0390707,37 / 947
0.0,1.0,1.0,6.0,12.0,5.0,0.0,8.0,6.0,983.0,0.0381605,"39 / 1,022"


Top-10 Hit Ratios: 


...

Top-10 Hit Ratios: 


k,hit_ratio
1,0.9634
2,0.9857
3,0.9930000
4,0.9960001
5,0.9976001
6,0.9986001
7,0.9995000
8,0.9997
9,0.9999
10,1.0


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
,2016-10-08 07:54:44,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan,nan,nan
,2016-10-08 07:54:46,1 min 4.667 sec,42828 obs/sec,1.6653,1,99918.0,0.2177593,0.1900343,0.0545931,0.2292818,0.2124221,0.06
,2016-10-08 07:54:54,1 min 12.239 sec,43851 obs/sec,6.6654833,4,399929.0,0.1640790,0.1062383,0.0313255,0.1947422,0.1604174,0.045
,2016-10-08 07:55:03,1 min 21.464 sec,44820 obs/sec,13.3258833,8,799553.0,0.1413583,0.0775775,0.0236704,0.1817095,0.1509310,0.0382
,2016-10-08 07:55:10,1 min 28.565 sec,45041 obs/sec,18.3333333,11,1100000.0,0.1302917,0.0645206,0.0209508,0.1775772,0.1538043,0.0366
,2016-10-08 07:55:17,1 min 35.480 sec,45405 obs/sec,23.3361333,14,1400168.0,0.1251970,0.0644976,0.0185334,0.1833011,0.1595036,0.0391
,2016-10-08 07:55:24,1 min 42.323 sec,45689 obs/sec,28.3343333,17,1700060.0,0.1183463,0.0571282,0.0167204,0.1820044,0.1641381,0.0393
,2016-10-08 07:55:29,1 min 47.088 sec,45809 obs/sec,31.6691167,19,1900147.0,0.1203134,0.0584976,0.0183320,0.1882238,0.1753694,0.0422
,2016-10-08 07:55:29,1 min 47.604 sec,45793 obs/sec,31.6691167,19,1900147.0,0.1302917,0.0645206,0.0209508,0.1775772,0.1538043,0.0366


deeplearning Model Build progress: |██████████████████████████████████████████████████████████| 100%


Specify a model and a file path. The default path is the current working directory.

In [99]:
model_path = h2o.save_model(
     model = model,
     #path = "/tmp/mymodel",
     force = True)

print model_path

/home/fmilo/workspace/h2o-deepwater/h2o-py/demos/Grid_DeepLearning_py_6_sid_88bf_model_python_1475860541731_17_model_1


In [105]:
!ls -lah $model_path

-rw-rw-r-- 1 fmilo fmilo 440K Oct  8 08:26 /home/fmilo/workspace/h2o-deepwater/h2o-py/demos/Grid_DeepLearning_py_6_sid_88bf_model_python_1475860541731_17_model_1


After restarting H2O, you can load the saved model by specifying the host and model file path. 

Note: The saved model must be the same version used to save the model.

In [106]:
# Load model from disk
saved_model = h2o.load_model(model_path)

You can also use the following commands to retrieve a model from its H2O key.
This is useful if you have created an H2O model using the web interface and
want to continue the modeling process in another language, for example **R**.

In [107]:
# Retrieve model by H2O key
model = h2o.get_model(model_id=model_checkpoint._id)

## Achieving 85% accuracy

In [112]:
model = H2ODeepLearningEstimator(
    activation="RectifierWithDropout",
    hidden=[1024,1024,2048],
    epochs=10,
    input_dropout_ratio=0.2, 
    l1=1e-5, 
    max_w2=10.0,
    train_samples_per_iteration=-1,
    classification_stop=-1,
    stopping_rounds=0
)

In [113]:
model.train(
    x=x, 
    y=y,
    training_frame=train_df,
    validation_frame=test_df
)

deeplearning Model Build progress: |██████████████████████████████████████████████████████████| 100%


## Scalability

![](../../h2o-docs/src/booklets/v2_2015/source/images/

In [115]:
!ls ../../h2o-docs/src/booklets/v2_2015/source/images/

mnistdigits.jpg  scurve.png  waves.png
